In [76]:
import pandas as pd
import numpy as np
import math

class NaiveBayesGaussian:
    def __init__(self):
        self.class_priors = None
        self.mean_var = None

    def fit(self, X_train, y_train):
        self.class_priors = {c: np.sum(y_train == c) / len(y_train) for c in np.unique(y_train)}

        self.mean_var = {}
        for c in np.unique(y_train):
            self.mean_var[c] = {}
            for feature in X_train.columns:
                self.mean_var[c][feature] = (X_train[y_train == c][feature].mean(), X_train[y_train == c][feature].var())

    def gaussian_prob(self, x, mean, var):
        coefficient = 1 / np.sqrt(2 * np.pi * var)
        exponential = np.exp(-(x - mean) ** 2 / (2 * var))
        return coefficient * exponential

    def predict(self, X):
        class_probs = {}
        for c in self.class_priors:
            class_probs[c] = self.class_priors[c]
            for feature in X.index:
                mean, var = self.mean_var[c][feature]
                class_probs[c] *= self.gaussian_prob(X[feature], mean, var)
        return max(class_probs, key=class_probs.get)

    def predict_batch(self, X_test):
        return X_test.apply(lambda x: self.predict(x), axis=1)


In [77]:
class Evaluation:
    @staticmethod
    def confusion_matrix(y_true, y_pred):
        unique_labels = np.unique(y_true)
        matrix = {label: {l: 0 for l in unique_labels} for label in unique_labels}

        for true, pred in zip(y_true, y_pred):
            matrix[true][pred] += 1

        return matrix

    @staticmethod
    def classification_report(y_true, y_pred):
        cm = Evaluation.confusion_matrix(y_true, y_pred)
        report = {}
        
        for label in cm:
            TP = cm[label][label]
            FP = sum(cm[l][label] for l in cm if l != label)
            FN = sum(cm[label][l] for l in cm if l != label)
            TN = sum(cm[l][k] for l in cm for k in cm[l] if l != label and k != label)
            
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            report[label] = {
                'precision': precision,
                'recall': recall,
                'f1_score': f1_score
            }

        return report

In [78]:
# Truyền dữ liệu đầu vào
def readDataInput(file_path, value_train_data, rd_state):
    
    if file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        data = pd.read_excel(file_path)
    else:
        raise ValueError("File nay khong ton tai")

    # Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
    train_data = data.sample(frac=value_train_data, random_state=rd_state) # Lấy ngẫu nhiên 80% phần tử có trong tập dữ liệu
    test_data = data.drop(train_data.index) # Lấy 20% còn lại và tránh trùng lặp với tập train

    # Tách dữ liệu tập huấn luyện
    X_train = train_data.drop('label', axis=1)
    y_train = train_data['label']

    # Tách dữ liệu tập kiểm tra
    X_test = test_data.drop('label', axis=1) 
    y_test = test_data['label']

    # Khởi tạo và huấn luyện mô hình
    model = NaiveBayesGaussian()
    model.fit(X_train, y_train)

    # Dự đoán trên tập test
    y_pred = model.predict_batch(X_test)

    # Tính độ chính xác
    accuracy = np.mean(y_pred == y_test)

    # Đánh giá mô hình
    conf_matrix = Evaluation.confusion_matrix(y_test, y_pred)
    report = Evaluation.classification_report(y_test, y_pred)

    print(f"Accuracy: {accuracy}") # Độ chính xác dựa vào số mẫu đã kiểm tra
    print("Classification Report:")
    for label, metrics in report.items():
        print(f"Class {label}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value}")
    print("Confusion Matrix:")
    for label in conf_matrix:
        print(f"Class {label}: {conf_matrix[label]}")


In [79]:
readDataInput('/kaggle/input/team2-dataset/team2_dataset/chap9.xlsx', 0.8, 1)

Accuracy: 0.7
Classification Report:
Class 0:
  precision: 0.4
  recall: 1.0
  f1_score: 0.5714285714285715
Class 1:
  precision: 1.0
  recall: 0.625
  f1_score: 0.7692307692307693
Confusion Matrix:
Class 0: {0: 2, 1: 0}
Class 1: {0: 3, 1: 5}
